# Broadband Upload Speeds in VT & NH

_By Carter Stowell, Feb 2019_  

This notebook provides a summary analysis of internet upload speeds by county across Vermont and New Hampshire.  

__Why look at upload speeds?__  

Businesses and entrepreneurs rely on strong upload and download speeds for basic operations and communications, such as video conferencing (Zoom, GoToMeeting, Google Hangouts, Skype), hard drive backups, cloud applications (Google Docs, Dropbox, iCloud), voice over IP (VoIP), telephone service, and attachments on outgoing email, to name a few. By contrast, home internet users care most about download speeds to satisfy typical consumer needs, such as watching movies on-demand. Lines blur in the case of remotely located employees, and businesses seeking workers and offices in well-connected areas.  

__How good is broadband in VT and NH overall with respect to other states?__  

In its 2018 analysis ["Internet Access Rankings"](https://www.usnews.com/news/best-states/rankings/infrastructure/internet-access) (U.S. News & World Report, 2018), Vermont and New Hampshire ranked 13th and 14th overall based on composite measures of broadband internet access. However, lower rankings of 22nd for VT and 34th for NH were given for access to ultra-fast internet of at least 1 gigabit speeds.  

__Which counties across VT and NH provide the best upload speeds?__  

To provide this analysis, the following script acquires, cleans and blends two data sources, one for county-level population data from the U.S. Census, and another for broadband availability, specifically Maximum Upload Speed by County. The output dataset(s) are used to develop mapping applications in CARTO. A derived measure of "Population-weighted upload speed" supports a per-capita comparison analysis by county, potentially useful when considering shared-bandwidth technologies like DSL.  

Weighted upload speeds can be calculated in various ways. Is a larger population better or worse? Should the weighted metric support comparisons between counties in different states, or only within a given state? The following are two of many possible weighting calculations. The first is a max upload speed (Mbps) per capita based on county population size (county_pop_size) which can be used to highlight counties with smaller populations and higher upload speeds. The second is based on max upload speed (Mbps) by the county percentage of state population (county_pop_pct) which would best support within-state but not between-state comparisons:

> $max\_upload\_wt = max\_upload / county\_pop\_size$

or

> $max\_upload\_wt\_pct = max\_upload / county\_pop\_pct$

Between these two, per capita weighting by population size is preferred for between-state comparisons of upload speeds relative to county populuations.  

**Goals & Significance.**  This small execerise could be expanded to a scoring algorithm(s) for identifying  locations based on multiple criteria. This could be enhanced by modeling covariate relationships that take into account measures of people and place--both objective and subjective--that can collectively represent opportunities for rural innovation.

## Data Sources

- [US Census American Community Survey (ACS)](https://www.census.gov/programs-surveys/acs) - 2017 Data Release
  - ACS 5-year, 2013-2017, for Population Size by County
- [FCC.gov Form 477](https://www.fcc.gov/general/broadband-deployment-data-fcc-form-477) - 2017 Broadband Data includes Max Advertised Upload Speeds by County
  - VT: https://transition.fcc.gov/form477/BroadbandData/Fixed/Dec17/Version%201/VT-Fixed-Dec2017.zip
  - NH: https://transition.fcc.gov/form477/BroadbandData/Fixed/Dec17/Version%201/NH-Fixed-Dec2017.zip

## Setup
__Import Python packages used for analysis__  

Learn more about the CensusData package for Python at https://pypi.org/project/CensusData/

In [1]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.precision', 2)

In [2]:
import censusdata  # interface to US Census data

## Import Census Data

Notes

- `33` is the state code for New Hampshire
- `50` is the state code for Vermont
- `050` is the summary level code for Counties. Read more in [CensusData package documentation](https://jtleider.github.io/censusdata/geographies.html).
- `B01001_001` is the table UniqueID for population size. Sourced from [Table Cells in ACS Summary docs](https://www.census.gov/programs-surveys/acs/technical-documentation/summary-file-documentation.html).

In [3]:
# Pull data on all VT county-level geographies from the ACS 2013-2017 5-year estimates
vtco = censusdata.download('acs5', 2017,
                             censusdata.censusgeo([('state', '50'), ('county', '*')]),
                             ['B01001_001E'])

# Pull data on all VT county-level geographies from the ACS 2013-2017 5-year estimates
nhco = censusdata.download('acs5', 2017,
                             censusdata.censusgeo([('state', '33'), ('county', '*')]),
                             ['B01001_001E'])

# rename column
vtco = vtco.rename(columns={'B01001_001E': 'population'})
nhco = nhco.rename(columns={'B01001_001E': 'population'})

In [4]:
# Show VT counties sorted by population
vtco.sort_values('population', ascending=False, inplace=True)
vtco.head(30)

,population
"Chittenden County, Vermont: Summary level: 050, state:50> county:007",160985
"Rutland County, Vermont: Summary level: 050, state:50> county:021",59676
"Washington County, Vermont: Summary level: 050, state:50> county:023",58691
"Windsor County, Vermont: Summary level: 050, state:50> county:027",55485
"Franklin County, Vermont: Summary level: 050, state:50> county:011",48816
"Windham County, Vermont: Summary level: 050, state:50> county:025",43332
"Addison County, Vermont: Summary level: 050, state:50> county:001",36825
"Bennington County, Vermont: Summary level: 050, state:50> county:003",36054
"Caledonia County, Vermont: Summary level: 050, state:50> county:005",30576
"Orange County, Vermont: Summary level: 050, state:50> county:017",28901


In [5]:
# Show NH counties sorted by population
nhco.sort_values('population', ascending=False, inplace=True)
nhco.head(30)

,population
"Hillsborough County, New Hampshire: Summary level: 050, state:33> county:011",406371
"Rockingham County, New Hampshire: Summary level: 050, state:33> county:015",302479
"Merrimack County, New Hampshire: Summary level: 050, state:33> county:013",147958
"Strafford County, New Hampshire: Summary level: 050, state:33> county:017",126552
"Grafton County, New Hampshire: Summary level: 050, state:33> county:009",89280
"Cheshire County, New Hampshire: Summary level: 050, state:33> county:005",76109
"Belknap County, New Hampshire: Summary level: 050, state:33> county:001",60383
"Carroll County, New Hampshire: Summary level: 050, state:33> county:003",47524
"Sullivan County, New Hampshire: Summary level: 050, state:33> county:019",43073
"Coos County, New Hampshire: Summary level: 050, state:33> county:007",32119


In [6]:
# Calculate total VT population as sum of counties
vt_totalpop = vtco.population.sum()  # total of counties
print('VT total population: ', format(vt_totalpop, ','))

VT total population:  624,636


In [7]:
# Calculate total NH population as sum of counties
nh_totalpop = nhco.population.sum()  # total of counties
print('NH total population: ', format(nh_totalpop, ','))

NH total population:  1,331,848


In [8]:
# Add columns with calculation of population percentage by county within each state
vtco['population_pct'] = vtco['population']/vt_totalpop
nhco['population_pct'] = nhco['population']/nh_totalpop

In [9]:
# Merge population dataframes usin full outer join 
#   reset_index() on dataframe sets integer index and moves location to new column
vtnh_pop = pd.merge(
    vtco.reset_index(),
    nhco.reset_index(),
    how='outer')

# rename column
vtnh_pop = vtnh_pop.rename(columns={'index': 'location'})

In [38]:
vtnh_pop.head(30)  # display merged dataframe

,location,population,population_pct
0,"Chittenden County, Vermont: Summary level: 050...",160985,2.58e-01
1,"Rutland County, Vermont: Summary level: 050, s...",59676,9.55e-02
2,"Washington County, Vermont: Summary level: 050...",58691,9.40e-02
3,"Windsor County, Vermont: Summary level: 050, s...",55485,8.88e-02
4,"Franklin County, Vermont: Summary level: 050, ...",48816,7.82e-02
5,"Windham County, Vermont: Summary level: 050, s...",43332,6.94e-02
6,"Addison County, Vermont: Summary level: 050, s...",36825,5.90e-02
7,"Bennington County, Vermont: Summary level: 050...",36054,5.77e-02
8,"Caledonia County, Vermont: Summary level: 050,...",30576,4.90e-02
9,"Orange County, Vermont: Summary level: 050, st...",28901,4.63e-02


In [11]:
# set index to location
#vtnh_pop.set_index('location')

### Import VT Broadband data

In [12]:
vt_fcc477 = pd.read_csv('https://transition.fcc.gov/form477/BroadbandData/Fixed/Dec17/Version%201/VT-Fixed-Dec2017.zip')
nh_fcc477 = pd.read_csv('https://transition.fcc.gov/form477/BroadbandData/Fixed/Dec17/Version%201/NH-Fixed-Dec2017.zip')

In [13]:
vt_fcc477.head()

,LogRecNo,Provider_Id,FRN,ProviderName,DBAName,HoldingCompanyName,HocoNum,HocoFinal,StateAbbr,BlockCode,TechCode,Consumer,MaxAdDown,MaxAdUp,Business,MaxCIRDown,MaxCIRUp
0,220639,34287,4335584,MCI Communications Corporation,MCI,Verizon Communications Inc.,131425,Verizon Communications Inc.,VT,500019603001003,30,0,0.0,0.0,1,1.50,1.50
1,220640,34287,4335584,MCI Communications Corporation,MCI,Verizon Communications Inc.,131425,Verizon Communications Inc.,VT,500019608002021,30,0,0.0,0.0,1,1.50,1.50
2,220641,34287,4335584,MCI Communications Corporation,MCI,Verizon Communications Inc.,131425,Verizon Communications Inc.,VT,500039704002021,30,0,0.0,0.0,1,1.50,1.50
3,220642,34287,4335584,MCI Communications Corporation,MCI,Verizon Communications Inc.,131425,Verizon Communications Inc.,VT,500059575001069,30,0,0.0,0.0,1,3.00,3.00
4,220643,34287,4335584,MCI Communications Corporation,MCI,Verizon Communications Inc.,131425,Verizon Communications Inc.,VT,500059575002021,30,0,0.0,0.0,1,0.77,0.77


**Block code deconstruction**  

Image is sourced from web page http://proximityone.com/geo_blocks.htm  

<img src="http://proximityone.com/cv_dr_graphics/geo_block.jpg">

In [14]:
# example county code
str(vt_fcc477['BlockCode'][0])[2:5]

'001'

In [15]:
# define lambda function to extract county code from block code
fn_state_code_in_block = lambda x: str(x)[0:2]  # 2-digit state code
fn_county_code_in_block = lambda x: str(x)[2:5]  # 3-digit county code
fn_state_county_in_block = lambda x: str(x)[0:5]  # combo

In [16]:
# add column with StateCode
vt_fcc477['StateCode'] = vt_fcc477['BlockCode'].apply(fn_state_code_in_block)
nh_fcc477['StateCode'] = nh_fcc477['BlockCode'].apply(fn_state_code_in_block)

# add column with CountyCode
vt_fcc477['CountyCode'] = vt_fcc477['BlockCode'].apply(fn_county_code_in_block)
nh_fcc477['CountyCode'] = nh_fcc477['BlockCode'].apply(fn_county_code_in_block)

# add column with CountyCode
vt_fcc477['StateCountyCode'] = vt_fcc477['BlockCode'].apply(fn_state_county_in_block)
nh_fcc477['StateCountyCode'] = nh_fcc477['BlockCode'].apply(fn_state_county_in_block)

In [17]:
nh_fcc477.head()

,LogRecNo,Provider_Id,FRN,ProviderName,DBAName,HoldingCompanyName,HocoNum,HocoFinal,StateAbbr,BlockCode,TechCode,Consumer,MaxAdDown,MaxAdUp,Business,MaxCIRDown,MaxCIRUp,StateCode,CountyCode,StateCountyCode
0,10360,34057,3784063,"MetroCast Cablevision of New Hampshire, LLC",Metrocast,Harron Communications LP,130591,Harron Communications LP,NH,330019651001005,42,1,105.0,10.0,1,150.0,10.0,33,001,33001
1,10361,34057,3784063,"MetroCast Cablevision of New Hampshire, LLC",Metrocast,Harron Communications LP,130591,Harron Communications LP,NH,330019651001006,42,1,105.0,10.0,1,150.0,10.0,33,001,33001
2,10362,34057,3784063,"MetroCast Cablevision of New Hampshire, LLC",Metrocast,Harron Communications LP,130591,Harron Communications LP,NH,330019651001007,42,1,105.0,10.0,1,150.0,10.0,33,001,33001
3,10363,34057,3784063,"MetroCast Cablevision of New Hampshire, LLC",Metrocast,Harron Communications LP,130591,Harron Communications LP,NH,330019651001008,42,1,105.0,10.0,1,150.0,10.0,33,001,33001
4,10364,34057,3784063,"MetroCast Cablevision of New Hampshire, LLC",Metrocast,Harron Communications LP,130591,Harron Communications LP,NH,330019651001013,42,1,105.0,10.0,1,150.0,10.0,33,001,33001


In [18]:
# Do VT county codes seem OK? How many uniques? Expecting 14 counties. OK
vt_fcc477['CountyCode'].describe()

count     183407
unique        14
top          027
freq       22306
Name: CountyCode, dtype: object

In [19]:
# Do NH county codes seem OK? How many uniques?  Expecting 10 counties. OK
nh_fcc477['CountyCode'].describe()

count     274631
unique        10
top          011
freq       56297
Name: CountyCode, dtype: object

In [20]:
# columns needed for upload analysis
select_cols = ['StateAbbr','StateCode','CountyCode','StateCountyCode','MaxAdUp','MaxCIRUp']

# dictionary of aggregations
aggregations = {
    'MaxAdUp': ['max'],
    'MaxCIRUp': ['max']  # agg fcn to perform
}

Data descriptions, excerpt from FCC's [Explanation of Broadband Deployment Data](https://www.fcc.gov/general/explanation-broadband-deployment-data)
- **Consumer**: (0/1) where 1 = Provider can or does offer consumer/mass market/residential service in the block
- **MaxAdDown**: Maximum advertised downstream speed/bandwidth offered by the provider in the block for Consumer service
- **MaxAdUp**: Maximum advertised upstream speed/bandwidth offered by the provider in the block for Consumer service
- **Business**: (0/1) where 1 = Provider can or does offer business/government service in the block
- **MaxCIRDown**: Maximum contractual downstream bandwidth offered by the provider in the block for Business service (filer directed to report 0 if the contracted service is sold on a "best efforts" basis without a guaranteed data-throughput rate)
- **MaxCIRUp**: Maximum contractual upstream bandwidth offered by the provider in the block for Business service (filer directed to report 0 if the contracted service is sold on a "best efforts" basis without a guaranteed data-throughput rate)

In [21]:
# new dataframe
vt_uploads = vt_fcc477[select_cols].groupby(['StateAbbr','StateCode','CountyCode','StateCountyCode']).agg(aggregations)
nh_uploads = nh_fcc477[select_cols].groupby(['StateAbbr','StateCode','CountyCode','StateCountyCode']).agg(aggregations)

In [22]:
vt_uploads   # MaxAdUp occasionally lower than MaxCIRUp

MaxAdUp MaxCIRUp
                                                   max      max
StateAbbr StateCode CountyCode StateCountyCode                 
VT        50        001        50001            1000.0   1000.0
                    003        50003            1000.0   1000.0
                    005        50005              40.0   1000.0
                    007        50007            1000.0   1000.0
                    009        50009              50.0    100.0
                    011        50011            1000.0   1000.0
                    013        50013            1000.0   1000.0
                    015        50015            1000.0   1000.0
                    017        50017             700.0   1000.0
                    019        50019              75.0   1000.0
                    021        50021            1000.0   1000.0
                    023        50023            1000.0   1000.0
                    025        50025            1000.0   1000.0
                    027        50027            1000.0   1000.0

In [23]:
nh_uploads   # MaxAdUp typically lower than MaxCIRUp

MaxAdUp MaxCIRUp
                                                   max      max
StateAbbr StateCode CountyCode StateCountyCode                 
NH        33        001        33001             120.0   1000.0
                    003        33003              75.0   1000.0
                    005        33005             200.0   1000.0
                    007        33007              75.0   1000.0
                    009        33009              50.0   1000.0
                    011        33011            1000.0   1000.0
                    013        33013             400.0   1000.0
                    015        33015            1000.0   1000.0
                    017        33017            1000.0   1000.0
                    019        33019             120.0   1000.0

In [24]:
# flatten hierarchical column index
# combine/join MultiIndex column names into one Index
vt_uploads.columns = ['_'.join(col).strip() for col in vt_uploads.columns.values]
nh_uploads.columns = ['_'.join(col).strip() for col in nh_uploads.columns.values]

In [25]:
# flatten hierarchical row index
vt_uploads = vt_uploads.reset_index()
nh_uploads = nh_uploads.reset_index()

In [26]:
vt_uploads

,StateAbbr,StateCode,CountyCode,StateCountyCode,MaxAdUp_max,MaxCIRUp_max
0,VT,50,001,50001,1000.0,1000.0
1,VT,50,003,50003,1000.0,1000.0
2,VT,50,005,50005,40.0,1000.0
3,VT,50,007,50007,1000.0,1000.0
4,VT,50,009,50009,50.0,100.0
5,VT,50,011,50011,1000.0,1000.0
6,VT,50,013,50013,1000.0,1000.0
7,VT,50,015,50015,1000.0,1000.0
8,VT,50,017,50017,700.0,1000.0
9,VT,50,019,50019,75.0,1000.0


In [27]:
nh_uploads

,StateAbbr,StateCode,CountyCode,StateCountyCode,MaxAdUp_max,MaxCIRUp_max
0,NH,33,001,33001,120.0,1000.0
1,NH,33,003,33003,75.0,1000.0
2,NH,33,005,33005,200.0,1000.0
3,NH,33,007,33007,75.0,1000.0
4,NH,33,009,33009,50.0,1000.0
5,NH,33,011,33011,1000.0,1000.0
6,NH,33,013,33013,400.0,1000.0
7,NH,33,015,33015,1000.0,1000.0
8,NH,33,017,33017,1000.0,1000.0
9,NH,33,019,33019,120.0,1000.0


In [28]:
# combine VT and NH into one dataframe
output = pd.merge(vt_uploads, nh_uploads, how='outer')

In [29]:
output

,StateAbbr,StateCode,CountyCode,StateCountyCode,MaxAdUp_max,MaxCIRUp_max
0,VT,50,001,50001,1000.0,1000.0
1,VT,50,003,50003,1000.0,1000.0
2,VT,50,005,50005,40.0,1000.0
3,VT,50,007,50007,1000.0,1000.0
4,VT,50,009,50009,50.0,100.0
5,VT,50,011,50011,1000.0,1000.0
6,VT,50,013,50013,1000.0,1000.0
7,VT,50,015,50015,1000.0,1000.0
8,VT,50,017,50017,700.0,1000.0
9,VT,50,019,50019,75.0,1000.0


## Appendix

#### County-Level Maps

- VT broadband availability maps, https://publicservice.vermont.gov/content/broadband-availability has links to three PDF map downloads based on Up/Down Mbps levels
- Broadband speeds and availability in the United States, http://www.governing.com/gov-data/broadband-speeds-availability.html (web-based, interactive map)

#### Potential data sources

- M-Lab (https://www.measurementlab.net/data/) is an open-source project jointly run by Google, Princeton University, and several other public entities. M-Lab data are used by BroadBandNow, as reported at https://broadbandnow.com/report/us-states-internet-coverage-speed-2018/

- OOKLA speed test data are used by the FCC, according to BroadBandNow. https://www.speedtest.net/reports/united-states/2018/fixed/

#### Useful Websites

- https://broadbandnow.com/Vermont
- https://broadbandnow.com/New-Hampshire
- [DSLreports.com - Good, Bad, Ugly](http://www.dslreports.com/gbu) for consumer ISP reviews of Satellite, Cable, Fiber, Mobile. For example, [Comcast Xfinity upload speeds by plan](http://www.dslreports.com/faq/15643).

#### Related Articles

- [*Internet Access Rankings*](https://www.usnews.com/news/best-states/rankings/infrastructure/internet-access) by U.S. News & World Report, 2018.
- [*Gov. Sununu signs broadband infrastructure bill into law*](https://www.sentinelsource.com/news/local/gov-sununu-signs-broadband-infrastructure-bill-into-law/article_4c07060c-d209-54fc-96fb-049ffc0a8a7e.html) by William Holt, Keene Sentinel, Jun 1, 2018
- [*Best Internet Plans & Providers in Vermont*](https://www.whistleout.com/Internet/Guides/best-internet-providers-in-vermont) by Ella Wagner, WhistleOut, Oct 19, 2018.
- [*How to compare Internet service providers — by upload speed*](https://www.usatoday.com/story/tech/columnist/2016/06/26/how-compare-internet-service-providers-upload-speed/86361172/) by Rob Pegoraro, Special for USA TODAY, June 26, 2016.


============================================================================

### Exploring examples from CensusData package

Based on example at https://jtleider.github.io/censusdata/example1.html

To download data, we need to identify the relevant tables containing the variables of interest to us. 

In [30]:
censusdata.search('acs5', 2017, 'label', 'unemploy')[62:65]

[('B23007_011E',
  'PRESENCE OF OWN CHILDREN UNDER 18 YEARS BY FAMILY TYPE BY EMPLOYMENT STATUS',
  'Estimate!!Total!!With own children under 18 years!!Married-couple family!!Husband in labor force!!Unemployed!!Wife in labor force'),
 ('B23007_012E',
  'PRESENCE OF OWN CHILDREN UNDER 18 YEARS BY FAMILY TYPE BY EMPLOYMENT STATUS',
  'Estimate!!Total!!With own children under 18 years!!Married-couple family!!Husband in labor force!!Unemployed!!Wife in labor force!!Employed or in Armed Forces'),
 ('B23007_013E',
  'PRESENCE OF OWN CHILDREN UNDER 18 YEARS BY FAMILY TYPE BY EMPLOYMENT STATUS',
  'Estimate!!Total!!With own children under 18 years!!Married-couple family!!Husband in labor force!!Unemployed!!Wife in labor force!!Unemployed')]

Once we have identified a table of interest, we can use censusdata.printtable to show all variables included in the table:

In [31]:
censusdata.printtable(censusdata.censustable('acs5', 2017, 'B23025'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B23025_001E  | EMPLOYMENT STATUS FOR THE POPU | !! Estimate Total                                        | int  
B23025_002E  | EMPLOYMENT STATUS FOR THE POPU | !! !! Estimate Total In labor force                      | int  
B23025_003E  | EMPLOYMENT STATUS FOR THE POPU | !! !! !! Estimate Total In labor force Civilian labor fo | int  
B23025_004E  | EMPLOYMENT STATUS FOR THE POPU | !! !! !! !! Estimate Total In labor force Civilian labor | int  
B23025_005E  | EMPLOYMENT STATUS FOR THE POPU | !! !! !! !! Estimate Total In labor force Civilian labor | int  
B23025_006E  | EMPLOYMENT STATUS FOR THE POPU | !! !! !! Estimate Total In labor force Armed Forces      | int  
B23025_007E  | EMPLOYMENT STATUS FOR THE POPU | !! !! Estimate Total Not in labor force      

After identifying relevant variables, we then need to identify the geographies of interest. We are interested in counties in VH and NH.  First we look for the geographic identifier (FIPS code) for each state, then the identifiers for counties.

In [32]:
censusdata.geographies(censusdata.censusgeo([('state', '*')]), 'acs5', 2017)

{'Puerto Rico': censusgeo((('state', '72'),)),
 'Alabama': censusgeo((('state', '01'),)),
 'Alaska': censusgeo((('state', '02'),)),
 'Arizona': censusgeo((('state', '04'),)),
 'Arkansas': censusgeo((('state', '05'),)),
 'California': censusgeo((('state', '06'),)),
 'Colorado': censusgeo((('state', '08'),)),
 'Connecticut': censusgeo((('state', '09'),)),
 'District of Columbia': censusgeo((('state', '11'),)),
 'Delaware': censusgeo((('state', '10'),)),
 'Florida': censusgeo((('state', '12'),)),
 'Georgia': censusgeo((('state', '13'),)),
 'Hawaii': censusgeo((('state', '15'),)),
 'Idaho': censusgeo((('state', '16'),)),
 'Illinois': censusgeo((('state', '17'),)),
 'Indiana': censusgeo((('state', '18'),)),
 'Iowa': censusgeo((('state', '19'),)),
 'Kansas': censusgeo((('state', '20'),)),
 'Kentucky': censusgeo((('state', '21'),)),
 'Maine': censusgeo((('state', '23'),)),
 'Louisiana': censusgeo((('state', '22'),)),
 'Massachusetts': censusgeo((('state', '25'),)),
 'Maryland': censusgeo((('s

__What are all county codes in Vermont?__

In [33]:
censusdata.geographies(censusdata.censusgeo([('state', '50'), ('county', '*')]), 'acs5', 2017)

{'Rutland County, Vermont': censusgeo((('state', '50'), ('county', '021'))),
 'Orleans County, Vermont': censusgeo((('state', '50'), ('county', '019'))),
 'Grand Isle County, Vermont': censusgeo((('state', '50'), ('county', '013'))),
 'Orange County, Vermont': censusgeo((('state', '50'), ('county', '017'))),
 'Chittenden County, Vermont': censusgeo((('state', '50'), ('county', '007'))),
 'Lamoille County, Vermont': censusgeo((('state', '50'), ('county', '015'))),
 'Windsor County, Vermont': censusgeo((('state', '50'), ('county', '027'))),
 'Franklin County, Vermont': censusgeo((('state', '50'), ('county', '011'))),
 'Caledonia County, Vermont': censusgeo((('state', '50'), ('county', '005'))),
 'Windham County, Vermont': censusgeo((('state', '50'), ('county', '025'))),
 'Bennington County, Vermont': censusgeo((('state', '50'), ('county', '003'))),
 'Addison County, Vermont': censusgeo((('state', '50'), ('county', '001'))),
 'Essex County, Vermont': censusgeo((('state', '50'), ('county',

__What are all county codes in New Hampshire?__

In [34]:
censusdata.geographies(censusdata.censusgeo([('state', '33'), ('county', '*')]), 'acs5', 2017)

{'Cheshire County, New Hampshire': censusgeo((('state', '33'), ('county', '005'))),
 'Merrimack County, New Hampshire': censusgeo((('state', '33'), ('county', '013'))),
 'Grafton County, New Hampshire': censusgeo((('state', '33'), ('county', '009'))),
 'Rockingham County, New Hampshire': censusgeo((('state', '33'), ('county', '015'))),
 'Belknap County, New Hampshire': censusgeo((('state', '33'), ('county', '001'))),
 'Sullivan County, New Hampshire': censusgeo((('state', '33'), ('county', '019'))),
 'Hillsborough County, New Hampshire': censusgeo((('state', '33'), ('county', '011'))),
 'Strafford County, New Hampshire': censusgeo((('state', '33'), ('county', '017'))),
 'Coos County, New Hampshire': censusgeo((('state', '33'), ('county', '007'))),
 'Carroll County, New Hampshire': censusgeo((('state', '33'), ('county', '003')))}

Now that we have identified the variables and geographies of interest, we can download the data using `censusdata.download` and compute variables for the percent unemployed and the percent with no high school degree:

In [35]:
cookbg = censusdata.download('acs5', 2015,
                             censusdata.censusgeo([('state', '17'), ('county', '031'), ('block group', '*')]),
                             ['B23025_003E', 'B23025_005E', 'B15003_001E', 'B15003_002E', 'B15003_003E',
                              'B15003_004E', 'B15003_005E', 'B15003_006E', 'B15003_007E', 'B15003_008E',
                              'B15003_009E', 'B15003_010E', 'B15003_011E', 'B15003_012E', 'B15003_013E',
                              'B15003_014E', 'B15003_015E', 'B15003_016E'])
cookbg['percent_unemployed'] = cookbg.B23025_005E / cookbg.B23025_003E * 100
cookbg['percent_nohs'] = (cookbg.B15003_002E + cookbg.B15003_003E + cookbg.B15003_004E
                          + cookbg.B15003_005E + cookbg.B15003_006E + cookbg.B15003_007E + cookbg.B15003_008E
                          + cookbg.B15003_009E + cookbg.B15003_010E + cookbg.B15003_011E + cookbg.B15003_012E
                          + cookbg.B15003_013E + cookbg.B15003_014E +
                          cookbg.B15003_015E + cookbg.B15003_016E) / cookbg.B15003_001E * 100
cookbg = cookbg[['percent_unemployed', 'percent_nohs']]
cookbg.describe()

,percent_unemployed,percent_nohs
count,3983.00,3984.00
mean,12.00,15.19
std,10.09,13.23
min,0.00,0.00
25%,4.86,4.75
50%,9.24,11.66
75%,16.28,22.46
max,91.86,77.43


Next, we show the 30 block groups in Cook County with the highest rate of unemployment, and the percent with no high school degree in those block groups.

In [36]:
cookbg.sort_values('percent_unemployed', ascending=False).head(30)

,percent_unemployed,percent_nohs
"Block Group 1, Census Tract 8357, Cook County, Illinois: Summary level: 150, state:17> county:031> tract:835700> block group:1",91.86,0.00
"Block Group 2, Census Tract 6805, Cook County, Illinois: Summary level: 150, state:17> county:031> tract:680500> block group:2",66.27,19.54
"Block Group 3, Census Tract 5103, Cook County, Illinois: Summary level: 150, state:17> county:031> tract:510300> block group:3",64.07,16.97
"Block Group 2, Census Tract 6809, Cook County, Illinois: Summary level: 150, state:17> county:031> tract:680900> block group:2",61.46,42.33
"Block Group 1, Census Tract 4913, Cook County, Illinois: Summary level: 150, state:17> county:031> tract:491300> block group:1",56.40,14.64
"Block Group 5, Census Tract 2315, Cook County, Illinois: Summary level: 150, state:17> county:031> tract:231500> block group:5",55.58,44.72
"Block Group 3, Census Tract 8346, Cook County, Illinois: Summary level: 150, state:17> county:031> tract:834600> block group:3",54.96,17.85
"Block Group 2, Census Tract 6706, Cook County, Illinois: Summary level: 150, state:17> county:031> tract:670600> block group:2",54.13,9.57
"Block Group 2, Census Tract 8386, Cook County, Illinois: Summary level: 150, state:17> county:031> tract:838600> block group:2",53.78,48.41
"Block Group 5, Census Tract 4910, Cook County, Illinois: Summary level: 150, state:17> county:031> tract:491000> block group:5",53.57,38.23


Finally, we show the correlation between these two variables across all Cook County block groups:

In [37]:
cookbg.corr()

,percent_unemployed,percent_nohs
percent_unemployed,1.00,0.29
percent_nohs,0.29,1.00


There is a weak correlation of 0.29 between percent unemployed and percent no high school.